In [ ]:
!pip install assemblyai
import os
import nltk
from google.colab import files
import assemblyai as aai
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.tokenize import sent_tokenize
import torch
import ipywidgets as widgets
from IPython.display import display, HTML
import spacy
import numpy as np
from sentence_transformers import SentenceTransformer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Download spaCy model
!python -m spacy download en_core_web_md

# Initialize AssemblyAI (You'll need to provide your own API key)
aai.settings.api_key = "90c7d1fa6df9457cafc70305b4205022"  # Replace with your actual API key

# Summarization Types Dictionary with target compression rates
SUMMARY_TYPES = {
    "Executive Summary": {"ratio": 0.15, "description": "A brief overview capturing main points (10-20% of original)"},
    "Detailed Summary": {"ratio": 0.35, "description": "Key points with supporting information (30-40% of original)"},
    "Educational Notes": {"ratio": 0.55, "description": "Comprehensive notes for study purposes (50-60% of original)"},
    "Paraphrased Version": {"ratio": 0.75, "description": "Rewritten in simpler or clearer terms (70-80% of original)"}
}

  Using cached assemblyai-0.37.0-py3-none-any.whl.metadata (29 kB)
Using cached assemblyai-0.37.0-py3-none-any.whl (44 kB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 14.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

# Load advanced models for better summarization
def load_summarization_models():
    """Load better models for summarization"""
    models = {}

    try:
        # Load BART large CNN model for abstractive summarization (better quality)
        print("Loading advanced summarization model...")
        model_name = "facebook/bart-large-cnn"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
        models["abstractive"] = summarizer
        print("Advanced summarization model loaded successfully!")

        # Load SpaCy for better text processing
        print("Loading NLP processing model...")
        nlp = spacy.load("en_core_web_md")
        models["nlp"] = nlp

        # Load sentence transformer for semantic similarity
        print("Loading semantic model...")
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        models["semantic"] = sentence_model

        return models
    except Exception as e:
        print(f"Error loading models: {str(e)}")
        # Fallback to simpler model
        try:
            model_name = "sshleifer/distilbart-cnn-6-6"
            summarizer = pipeline("summarization", model=model_name)
            models["abstractive"] = summarizer
            print("Fallback summarization model loaded.")
            return models
        except:
            print("Could not load summarization models. Will use extractive summarization only.")
            return models


In [ ]:

# Improved extractive summarization using semantic analysis
def improved_extractive_summary(text, ratio=0.3, models=None):
    """Create a better extractive summary using NLP techniques and semantic similarity"""
    if not text or len(text) < 100:
        return "Text too short to summarize."

    try:
        # Use spaCy for better text processing if available
        if models and "nlp" in models:
            nlp = models["nlp"]
            doc = nlp(text)

            # Extract sentences and filter out very short ones
            sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 20]
        else:
            # Fallback to NLTK
            sentences = sent_tokenize(text)

        num_sentences = max(3, int(len(sentences) * ratio))

        if len(sentences) <= num_sentences:
            return text

        # Use semantic similarity for better sentence selection if available
        if models and "semantic" in models:
            semantic_model = models["semantic"]

            # Compute sentence embeddings
            sentence_embeddings = semantic_model.encode(sentences)

            # Compute document centroid
            centroid = np.mean(sentence_embeddings, axis=0)

            # Compute similarity to centroid
            similarities = []
            for i, embedding in enumerate(sentence_embeddings):
                similarity = np.dot(embedding, centroid) / (np.linalg.norm(embedding) * np.linalg.norm(centroid))
                similarities.append((i, similarity))

            # Get top sentences by similarity
            top_sentences = sorted(similarities, key=lambda x: x[1], reverse=True)[:num_sentences]
            top_sentences = sorted(top_sentences, key=lambda x: x[0])  # Sort by position
        else:
            # Fallback to frequency-based extraction
            from nltk.corpus import stopwords
            from nltk.tokenize import word_tokenize
            from nltk.probability import FreqDist

            # Remove stopwords and tokenize
            stop_words = set(stopwords.words('english'))
            words = word_tokenize(text.lower())
            words = [word for word in words if word.isalnum() and word not in stop_words]

            # Calculate word frequencies
            freq = FreqDist(words)

            # Score sentences based on word frequency
            sentence_scores = {}
            for i, sentence in enumerate(sentences):
                score = 0
                words_in_sentence = word_tokenize(sentence.lower())
                for word in words_in_sentence:
                    if word in freq:
                        score += freq[word]
                sentence_scores[i] = score / max(1, len(words_in_sentence))

            # Get top sentences
            top_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:num_sentences]
            top_sentences = sorted(top_sentences, key=lambda x: x[0])  # Sort by position

        summary = ' '.join([sentences[i] for i, _ in top_sentences])
        return summary
    except Exception as e:
        print(f"Error in extractive summarization: {str(e)}")
        return "Error generating summary. Please try again."


In [ ]:

# Generate a summary using the best available method
def generate_better_summary(text, summary_type="Detailed Summary", models=None):
    """Generate a high-quality summary of the provided text based on selected type"""
    if not text or len(text) < 100:
        return "Text too short to summarize."

    # Get ratio for the requested summary type
    ratio = SUMMARY_TYPES.get(summary_type, SUMMARY_TYPES["Detailed Summary"])["ratio"]

    # Use abstractive summarization if model is available
    if models and "abstractive" in models:
        summarizer = models["abstractive"]

        # Process text in chunks if it's too long
        if len(text) > 1024:
            # Split into semantic chunks using spaCy if available
            if "nlp" in models:
                nlp = models["nlp"]
                doc = nlp(text)

                # Create semantic chunks based on paragraphs/sections
                chunks = []
                current_chunk = ""

                for sent in doc.sents:
                    if len(current_chunk) + len(sent.text) < 1000:
                        current_chunk += " " + sent.text
                    else:
                        chunks.append(current_chunk.strip())
                        current_chunk = sent.text

                if current_chunk:
                    chunks.append(current_chunk.strip())
            else:
                # Fallback to simple sentence chunking
                sentences = sent_tokenize(text)
                chunks = []
                current_chunk = ""

                for sentence in sentences:
                    if len(current_chunk) + len(sentence) < 1000:
                        current_chunk += " " + sentence
                    else:
                        chunks.append(current_chunk.strip())
                        current_chunk = sentence

                if current_chunk:
                    chunks.append(current_chunk.strip())

            # Summarize each chunk based on the summary type
            chunk_summaries = []
            for chunk in chunks:
                try:
                    # Adjust length based on summary type
                    chunk_max_length = min(int(len(chunk) * ratio * 1.2), 250)  # Allow slightly longer for better quality
                    chunk_min_length = max(30, int(chunk_max_length * 0.6))

                    summary = summarizer(
                        chunk,
                        max_length=chunk_max_length,
                        min_length=chunk_min_length,
                        do_sample=False,
                        num_beams=4,  # Better beam search for quality
                        early_stopping=True
                    )[0]['summary_text']

                    chunk_summaries.append(summary)
                except Exception as e:
                    print(f"Error in chunk summarization, falling back: {str(e)}")
                    # Fallback to extractive for this chunk
                    chunk_summary = improved_extractive_summary(chunk, ratio, models)
                    chunk_summaries.append(chunk_summary)

            # Combine chunk summaries
            combined_summary = " ".join(chunk_summaries)

            # If the combined summary is still too long, summarize it once more
            if len(combined_summary) > 1500 and summary_type in ["Executive Summary", "Detailed Summary"]:
                try:
                    final_max_length = min(int(len(combined_summary) * ratio), 600)
                    final_min_length = max(100, int(final_max_length * 0.5))

                    final_summary = summarizer(
                        combined_summary,
                        max_length=final_max_length,
                        min_length=final_min_length,
                        do_sample=False,
                        num_beams=4
                    )[0]['summary_text']

                    return final_summary
                except:
                    return combined_summary
            else:
                return combined_summary
        else:
            # For shorter texts, summarize directly
            try:
                max_length = min(int(len(text) * ratio * 1.2), 600)
                min_length = max(50, int(max_length * 0.6))

                return summarizer(
                    text,
                    max_length=max_length,
                    min_length=min_length,
                    do_sample=False,
                    num_beams=4,
                    early_stopping=True
                )[0]['summary_text']
            except Exception as e:
                print(f"Error in abstractive summarization, falling back: {str(e)}")

    # Fallback to improved extractive summarization
    return improved_extractive_summary(text, ratio, models)

In [ ]:

# Post-process summary to improve quality
def post_process_summary(summary):
    """Clean up and improve the generated summary"""
    if not summary:
        return summary

    # Fix spacing issues
    summary = ' '.join(summary.split())

    # Ensure summary ends with proper punctuation
    if summary and summary[-1] not in ['.', '!', '?']:
        summary += '.'

    # Remove redundant sentence beginnings like "This text" or "The text"
    if summary.startswith("This text") or summary.startswith("The text"):
        sentences = sent_tokenize(summary)
        if len(sentences) > 1:
            summary = ' '.join(sentences[1:])

    return summary


In [ ]:

# Display interactive widgets for selecting summary type
def show_improved_summary_options(transcript, models=None):
    """Display interactive widgets for selecting and generating summary"""

    # Create dropdown for summary type selection
    summary_dropdown = widgets.Dropdown(
        options=list(SUMMARY_TYPES.keys()),
        value="Detailed Summary",
        description="Summary Type:",
        style={'description_width': 'initial'}
    )

    # Display descriptions for each summary type
    description_output = widgets.Output()
    with description_output:
        display(HTML(f"<p><b>{summary_dropdown.value}:</b> {SUMMARY_TYPES[summary_dropdown.value]['description']}</p>"))

    # Create button to generate summary
    generate_button = widgets.Button(
        description="Generate Summary",
        button_style="primary"
    )

    # Output area for the summary
    summary_output = widgets.Output()

    # Update description when dropdown changes
    def on_dropdown_change(change):
        description_output.clear_output()
        with description_output:
            display(HTML(f"<p><b>{change['new']}:</b> {SUMMARY_TYPES[change['new']]['description']}</p>"))

    summary_dropdown.observe(on_dropdown_change, names='value')

    # Function to handle button click
    def on_button_click(b):
        summary_output.clear_output()
        with summary_output:
            print("Generating summary...")
            summary = generate_better_summary(transcript, summary_dropdown.value, models)
            summary = post_process_summary(summary)

            print("\n=== SUMMARY ===")
            print(summary)

            # Calculate approximate percentage of original
            orig_word_count = len(transcript.split())
            summary_word_count = len(summary.split())
            percentage = round((summary_word_count / orig_word_count) * 100)

            print(f"\n(Summary is approximately {percentage}% of the original text)")

    generate_button.on_click(on_button_click)

    # Display all widgets
    display(widgets.VBox([
        widgets.HTML("<h3>Select Summary Type</h3>"),
        summary_dropdown,
        description_output,
        generate_button,
        summary_output
    ]))

In [ ]:

# Main function to transcribe audio and show summary options
def transcribe_and_summarize():
    #"""Upload and transcribe an audio file, then show improved summary options"""
    # Load advanced models
    print("Loading advanced summarization models...")
    models = load_summarization_models()

    # Upload an audio file
    print("Please upload an audio file:")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. You can also provide text directly.")
        text_input = widgets.Textarea(
            value='',
            placeholder='Paste your text here to summarize...',
            description='Text:',
            disabled=False,
            layout=widgets.Layout(width='90%', height='200px')
        )

        submit_button = widgets.Button(
            description="Process Text",
            button_style="primary"
        )

        output_area = widgets.Output()

        def on_submit(b):
            text = text_input.value
            if text.strip():
                with output_area:
                    print("\n=== ORIGINAL TEXT ===")
                    print(text)
                    show_improved_summary_options(text, models)

        submit_button.on_click(on_submit)
        display(widgets.VBox([text_input, submit_button, output_area]))
        return

    # Get the file path
    file_path = list(uploaded.keys())[0]

    print(f"Transcribing {file_path}...")
    transcriber = aai.Transcriber()

    try:
        # Transcribe the audio
        transcript = transcriber.transcribe(file_path)

        if hasattr(transcript, 'status') and transcript.status == aai.TranscriptStatus.error:
            print(f"Transcription error: {transcript.error}")
            return

        # Display the transcript
        print("\n=== TRANSCRIPT ===")
        print(transcript.text)

        # Show improved summary options
        print("\nChoose your summary type:")
        show_improved_summary_options(transcript.text, models)

        return {"transcript": transcript.text}

    except Exception as e:
        print(f"Error during transcription: {str(e)}")
        return None

# Simple function to directly summarize text (without UI)
def quick_summarize(text, summary_type="Detailed Summary"):
    """Quickly summarize text without UI components"""
    print("Loading models...")
    models = load_summarization_models()

    print(f"Generating {summary_type}...")
    summary = generate_better_summary(text, summary_type, models)
    summary = post_process_summary(summary)

    print("\n=== SUMMARY ===")
    print(summary)

    # Calculate approximate percentage of original
    orig_word_count = len(text.split())
    summary_word_count = len(summary.split())
    percentage = round((summary_word_count / orig_word_count) * 100)
    print(f"\n(Summary is approximately {percentage}% of the original text)")

    return summary

transcribe_and_summarize()



Loading advanced summarization models...
Loading advanced summarization model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Please upload an audio file:


Saving WhatsApp Audio 2025-04-08 at 21.41.54.opus to WhatsApp Audio 2025-04-08 at 21.41.54.opus
Transcribing WhatsApp Audio 2025-04-08 at 21.41.54.opus...

=== TRANSCRIPT ===
Hello. Hello. Mic testing. Mic testing. Okay, thank you.

Choose your summary type:


{'transcript': 'Hello. Hello. Mic testing. Mic testing. Okay, thank you.'}